# Mapping Statewide Stations 
Demoing how to approach the GTFS Data warehouse

In [2]:
import pandas as pd
import geopandas as gpd
import tqdm

In [4]:
# remember to set project at Gcloud command line level to `cal-itp-data-infra` otherwise this will fail
import gcsfs
fs = gcsfs.GCSFileSystem()

In [27]:
df = pd.read_csv('https://docs.google.com/spreadsheets/d/1qr49azk6p30mp96_7myKoO-Bb_bXMMn5ZzgbL-uPiPw/gviz/tq?tqx=out:csv&sheet=Data')

In [15]:

import ipywidgets  as widgets
import IPython.display
from ipywidgets import interact

In [21]:
options = fs.ls('gtfs-data/schedule')
widgets.Dropdown(
    options=options,
    description='Number:',
    disabled=False,
)

Dropdown(description='Number:', options=('gtfs-data/schedule/2021-02-19T00:00:00+00:00', 'gtfs-data/schedule/2…

In [30]:
def map_function(date):
    df = pd.read_csv('https://docs.google.com/spreadsheets/d/1qr49azk6p30mp96_7myKoO-Bb_bXMMn5ZzgbL-uPiPw/gviz/tq?tqx=out:csv&sheet=Data')
    stops = {}
    for agency_path in fs.ls(date):
        itp_id = agency_path.split('/')[-1]
        stop = pd.read_csv(f'gcs://{agency_path}/stops.txt')    
        stops[itp_id] = stop
    stops_named = {}
    for agency in stops:
        name = df[df.ITP_ID == int(agency)]['Agency Name'].values[0]
        stops_named[name] = stops[agency]
    dfs = []
    for agency in stops_named:
        t_df = stops_named[agency][['stop_lat','stop_lon']].assign(agency = agency)
        dfs.append(t_df)
    df2 = pd.concat(dfs)
    gdf = gpd.GeoDataFrame(
        df, geometry=gpd.points_from_xy(df.stop_lon, df.stop_lat))
    # Define a layer to display on a map
    layer = pdk.Layer(
        'HexagonLayer',
        df2,
        get_position=['stop_lon', 'stop_lat'],
        auto_highlight=True,
        elevation_scale=50,
        pickable=True,
        elevation_range=[0, 3000],
        extruded=True,                 
        coverage=1)

    # Set the viewport location
    view_state = pdk.ViewState(
        longitude=-119.4179,
        latitude=36.7783,
        zoom=6,
        min_zoom=5,
        max_zoom=15,
        pitch=40.5,
        bearing=-27.36)

    # Render
    r = pdk.Deck(layers=[layer], initial_view_state=view_state)
    return display(r.show())

In [5]:
stops = {}
for agency_path in fs.ls('gtfs-data/schedule/2021-03-21T00:00:00+00:00/'):
    itp_id = agency_path.split('/')[-1]
    stop = pd.read_csv(f'gcs://{agency_path}/stops.txt')    
    stops[itp_id] = stop

In [8]:
stops_named = {}
for agency in stops:
    name = df[df.ITP_ID == int(agency)]['Agency Name'].values[0]
    stops_named[name] = stops[agency]

In [9]:
dfs = []
for agency in stops_named:
    t_df = stops_named[agency][['stop_lat','stop_lon']].assign(agency = agency)
    dfs.append(t_df)

In [10]:
df = pd.concat(dfs)
gdf = gpd.GeoDataFrame(
    df, geometry=gpd.points_from_xy(df.stop_lon, df.stop_lat))

In [11]:
import pydeck as pdk

# Define a layer to display on a map
layer = pdk.Layer(
    'HexagonLayer',
    df,
    get_position=['stop_lon', 'stop_lat'],
    auto_highlight=True,
    elevation_scale=50,
    pickable=True,
    elevation_range=[0, 3000],
    extruded=True,                 
    coverage=1)

# Set the viewport location
view_state = pdk.ViewState(
    longitude=-119.4179,
    latitude=36.7783,
    zoom=6,
    min_zoom=5,
    max_zoom=15,
    pitch=40.5,
    bearing=-27.36)

# Render
r = pdk.Deck(layers=[layer], initial_view_state=view_state)
r.show()

DeckGLWidget(carto_key=None, custom_libraries=[], google_maps_key=None, json_input='{"initialViewState": {"bea…